# Big Data project

In [1]:
from pyspark.sql import SparkSession

## Start spaek session

In [2]:
spark = SparkSession.builder \
    .appName("Read E-commerce Datasets") \
    .getOrCreate()

25/05/17 03:33:32 WARN Utils: Your hostname, eltras resolves to a loopback address: 127.0.1.1; using 192.168.1.3 instead (on interface wlp2s0)
25/05/17 03:33:32 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/05/17 03:33:33 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


## get datasets

In [3]:
eshop_df = spark.read.option("header", "true") \
                     .option("sep", ";") \
                     .csv("hdfs://localhost:9000/Bigdatasets/e-shop-clothing-2008.csv")

print("e-shop-clothing-2008.csv")
eshop_df.show(5)
eshop_df.printSchema()

# ===== Dataset 2: Online Retail II =====
retail2_df = spark.read.option("header", "true") \
                       .csv("hdfs://localhost:9000/Bigdatasets/online_retail_II.csv")

print("online_retail_II.csv")
retail2_df.show(5)
retail2_df.printSchema()

# ===== Dataset 3: Online Retail =====
retail_df = spark.read.option("header", "true") \
                      .csv("hdfs://localhost:9000/Bigdatasets/Online-Retail.csv")

print("Online-Retail.csv")
retail_df.show(5)
retail_df.printSchema()

e-shop-clothing-2008.csv
+----+-----+---+-----+-------+----------+----------------------+-----------------------+------+--------+-----------------+-----+-------+----+
|year|month|day|order|country|session ID|page 1 (main category)|page 2 (clothing model)|colour|location|model photography|price|price 2|page|
+----+-----+---+-----+-------+----------+----------------------+-----------------------+------+--------+-----------------+-----+-------+----+
|2008|    4|  1|    1|     29|         1|                     1|                    A13|     1|       5|                1|   28|      2|   1|
|2008|    4|  1|    2|     29|         1|                     1|                    A16|     1|       6|                1|   33|      2|   1|
|2008|    4|  1|    3|     29|         1|                     2|                     B4|    10|       2|                1|   52|      1|   1|
|2008|    4|  1|    4|     29|         1|                     2|                    B17|     6|       6|                2| 